# Сбор автобусного маршрута по региону по остановкам (в случае если нет данных о маршрутах общественного транспорта)

In [1]:
# импорт модулей и библиотек

from shapely import Point
import pandas as pd
from tqdm.auto import tqdm
import geopandas as gpd
from dongraphio.utils.geometry_utils import clusterize_kmeans_geo_points

import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely.wkt import loads as from_wkt

In [ ]:
# CRS (в формате ESPG) и OSM ID (id полигона региона, id точки админ. центра региона) по тестовым регионам

# Ленинградская_область EPSG: 32636  / 176095 / N27025179 (Гатчина)
# Санкт_Петербург EPSG:32636 / 337422 / N27490597
# Москва EPSG:32637 / 102269 / N1686293227
# Волгоградская_область EPSG:32638 / 77665 /  N27504363
# Тульская_область EPSG:32637 /  81993 / N34389350
# Омская_область EPSG:32643 / 140292 /  N27503946
# Краснодарский_край EPSG:32637 / 108082 / N27505129
# Тюменская_область EPSG:32642 / 140291 / N27505666
# Московская_область EPSG:32637 / 51490 / N1686293227

Разделяем слой с остановками на уникальные маршруты, поскольку есть маршруты в разных областях ЛО c одинаковыми названиями, необходимо применить кластеризацию по расположению с искуственным шумом. Если все маршруты уникальные это не требуется

In [2]:
# импорт дорожного графа с ОСМ

from dongraphio.base_models.grapher_logic import BuildsGrapher
import networkx as nx

city_osm_id = 51490  # Фактический OSM ID города
city_crs = 3857     

grapher = BuildsGrapher(city_osm_id=city_osm_id, city_crs=city_crs)

network_type = "drive_service"
truncate_by_edge = True

drive_graph: nx.MultiDiGraph = grapher.get_osmnx_graph(network_type, truncate_by_edge)

2024-07-26 16:35:39.107 | DEBUG    | dongraphio.base_models.grapher_logic:get_osmnx_graph:94 - Extracting and preparing drive_service graph from OSM ...
/Users/sashamorozov/Documents/Code/don/dongraphio/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 24 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [3]:
def add_geometry_if_none(graph):
    for u, v, key, data in graph.edges(keys=True, data=True):
        if data.get('geometry') is None:
            x1, y1 = graph.nodes[u]['x'], graph.nodes[u]['y']
            x2, y2 = graph.nodes[v]['x'], graph.nodes[v]['y']
            data['geometry'] = LineString([(x1, y1), (x2, y2)])
    return graph

drive_graph = add_geometry_if_none(drive_graph)

In [4]:
# создание буфера дорог, для проецирования остановок
from shapely import from_wkt
import networkx as nx
from dongraphio.utils.graph_utils import buffer_directed_roads, nx_to_gdf

for i in drive_graph.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
drive_graph_gdf = nx_to_gdf(drive_graph, nodes=False, edges=True).to_crs(3857)
drive_graph_gdf_buffer = buffer_directed_roads(drive_graph_gdf, 3).to_crs(3857)


In [5]:
# загрузка автобусных остановок

dev_rout = gpd.read_file("/Users/sashamorozov/Downloads/moscow_region_bus_stops.geojson").to_crs(3857)

In [6]:
# преообразование остановок в лист

t =[]
dev_rout.geometry.apply(lambda x: t.append(x))
dev_rout1 = pd.DataFrame(data=[[t]])
devided_routes=dev_rout1
devided_routes

,0
0,"[POINT (4148671.2359044584 7502303.740831938),..."


In [7]:
len(devided_routes[0][0])

19153

In [8]:
# %load_ext autoreload
# %autoreload 2
from dongraphio.utils.graph_utils import convert_multidigraph_to_digraph, project_points_on_graph, \
    add_projected_points_as_nodes
from dongraphio.utils.tsp_solver import resolve_tsp
from dongraphio.utils.matrix_utils import get_dist_matrix_for_tsp
from dongraphio.utils.graph_utils import subgraph_by_path_for_single_route
import networkx as nx

city_crs = 3857
# bus_routes = nx.DiGraph()
error_routes = []
geometry_list = devided_routes[0].values.tolist()[0]
row_gdf = gpd.GeoDataFrame(data={"label": [1 for _ in geometry_list], "geometry": geometry_list})
# row_gdf = row_gdf.set_geometry('geometry')
row_gdf = row_gdf.set_crs(3857)
row_gdf = row_gdf.to_crs(city_crs)

projected_points = project_points_on_graph(drive_graph_gdf, drive_graph_gdf_buffer, row_gdf)

route_nodes, graph_with_new_points = add_projected_points_as_nodes(projected_points, drive_graph)

graph_with_new_points = convert_multidigraph_to_digraph(graph_with_new_points)

distance_matrix = get_dist_matrix_for_tsp_single(graph_with_new_points, route_nodes)

path = resolve_tsp(distance_matrix,240) 

route_nx = subgraph_by_path_for_single_route(path, distance_matrix, graph_with_new_points)

In [11]:
distance_matrix

,100044_1,100228_1,100937_1,100954_1,100971_1,101002_1,101253_1,10142_1,10143_1,10148_1,...,99046_1,99127_1,99183_1,99189_1,99324_1,99364_1,99671_1,99699_1,99762_1,99785_1
100044_1,0.0,16780.856705,58664.941477,41377.501219,29407.755283,46942.950186,9741.44829,545646.166488,545178.796188,544872.472138,...,39302.165982,38894.37098,30676.762036,35905.136251,40241.290569,39546.06512,24510.520604,27738.213972,33730.560659,24744.510779
100228_1,16765.018065,0.0,73915.112053,56627.671794,44657.925858,63667.996762,26466.494865,531614.806064,531147.435764,530841.111713,...,54552.336557,54144.541556,45926.932612,51155.306827,55491.461145,54796.235695,19131.90118,28520.432548,29896.061235,19365.891355
100937_1,49408.368655,63852.912871,0.0,26543.568385,21977.952449,71267.251353,55211.856456,594831.807655,594364.437355,594058.113304,...,13844.333148,15724.794147,20723.717203,14704.333418,12756.978736,14640.609286,45349.932771,39997.430139,45989.776826,44622.125945
100954_1,40051.027029,54495.571244,34473.667017,0.0,12620.610822,69945.353726,45854.514829,584040.120028,583572.749728,583266.425677,...,15110.891521,14703.09652,11366.375576,11713.861791,16050.016109,15354.790659,35992.591144,30640.088512,36632.435199,35264.784319
100971_1,28410.675254,42855.219469,31590.943242,14303.502984,0.0,60691.668951,34214.163055,573834.114253,573366.743953,573060.419902,...,12228.167747,11820.372745,3602.763801,8831.138016,13167.292334,12472.066884,24352.239369,18999.736737,24992.083424,23624.432544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99364_1,37721.157793,52165.702009,22654.545781,14856.357523,10290.741587,61125.70049,43524.645594,583144.596792,582677.226492,582370.902442,...,778.176286,4897.318284,9036.50634,3017.122555,5042.364873,0.0,33662.721908,28310.219276,34302.565963,32934.915083
99671_1,24006.169324,18643.38854,55560.937312,38273.497054,26303.751118,70909.148021,33707.646125,550032.138323,549564.768023,549258.443973,...,36198.161817,35790.366815,27572.757872,32801.132086,37137.286404,36442.060955,0.0,10166.257807,11541.886494,1011.716614
99699_1,26470.544361,24604.183577,49540.44835,32253.008091,20283.262155,69475.534059,32274.032162,555992.933361,555525.563061,555219.23901,...,30177.672854,29769.877853,21552.268909,26780.643124,31116.797442,30421.571992,6101.203477,0.0,6741.047532,5373.396652
99762_1,32788.120438,29702.109654,55858.024427,38570.584168,26600.838232,75793.110136,38591.608239,561090.859438,560623.489138,560317.165087,...,36495.248931,36087.45393,27869.844986,33098.219201,37434.373519,36739.148069,11199.129554,10463.344922,0.0,10471.322729


In [13]:
(list(route_nx.edges(data=True)))

[('100044_1',
  '105988_1',
  {'geometry': <MULTILINESTRING ((4414612.03 5757197.27, 4414612.75 5757196.8), (4414612.75...>,
   'route': '1',
   'length_meter': 385.3966889283349}),
 ('105988_1',
  '101624_1',
  {'geometry': <MULTILINESTRING ((4414658.77 5757187.84, 4414650.45 5757172.41), (4414650.4...>,
   'route': '1',
   'length_meter': 4483.739641571996}),
 ('101624_1',
  '105079_1',
  {'geometry': <MULTILINESTRING ((4417090.85 5755088.32, 4417057.3 5754987.33), (4417057.3 ...>,
   'route': '1',
   'length_meter': 2595.1052442721075}),
 ('105079_1',
  '101253_1',
  {'geometry': <MULTILINESTRING ((4417122.91 5755082.32, 4417134.96 5755117.5, 4417157.21 5...>,
   'route': '1',
   'length_meter': 8645.414353606038}),
 ('101253_1',
  '90580_1',
  {'geometry': <MULTILINESTRING ((4422777.63 5750328.42, 4422775.07 5750347.4), (4422775.07...>,
   'route': '1',
   'length_meter': 24297.70375177053}),
 ('90580_1',
  '85861_1',
  {'geometry': <MULTILINESTRING ((4435777.56 5733556.6, 4435714.

In [15]:
route_nx_copy = route_nx.copy()
route_nx_copy.graph['crs'] = 'EPSG:3857'

In [21]:
route_nx_copy_multi = nx.MultiDiGraph()
route_nx_copy_multi.add_nodes_from(route_nx_copy.nodes(data=True))
for u, v, data in route_nx_copy.edges(data=True):
    route_nx_copy_multi.add_edge(u, v, **data)

route_nx_copy_multi.graph['crs'] = 'EPSG:3857'

In [22]:
n, e = ox.graph_to_gdfs(route_nx_copy_multi)

In [26]:
n.explore()

In [25]:
for u, v, data in route_nx_copy_multi.edges(data=True):
    if 'geometry' in data:
        data['geometry'] = data['geometry'].wkt

nx.write_graphml(route_nx_copy_multi, 'krasnodar_bus_route_3857.graphml')

# Объединение графа маршрута с пешеходным графом

In [ ]:
from pyproj import Transformer
from shapely.geometry import Point, LineString,MultiLineString
import momepy
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiLineString

In [ ]:
local_crs = 32637

bus = nx.read_graphml('krasnodar_bus_route_3857.graphml')
bus = nx.MultiDiGraph(bus)

inter1 = nx.read_graphml('Краснодарский_край_inter.graphml')
walk_edges = [(e1, e2, key) for e1, e2, key, data in inter1.edges(keys=True, data=True) if data.get('type') == 'walk']
walk_subgraph = inter1.edge_subgraph(walk_edges)
walk = nx.MultiDiGraph(walk_subgraph)

In [ ]:
transformer = Transformer.from_crs("EPSG:3857", local_crs, always_xy=True)

def transform_point(point):
    x, y = transformer.transform(point.x, point.y)
    return Point(x, y)

def transform_geometry(geometry):
    if isinstance(geometry, Point):
        return transform_point(geometry)
    elif isinstance(geometry, LineString):
        return LineString([transform_point(Point(coord)) for coord in geometry.coords])
    elif isinstance(geometry, MultiLineString):
        return MultiLineString([LineString([transform_point(Point(coord)) for coord in line.coords]) for line in geometry.geoms])
    else:
        raise TypeError("Unsupported geometry type")

In [ ]:
# transform bus crs and convert geometry
for n, d in bus.nodes(data=True):
    point = Point(d['x'], d['y'])
    transformed_point = transform_point(point)
    bus.nodes[n]['x'] = transformed_point.x
    bus.nodes[n]['y'] = transformed_point.y
    bus.nodes[n]['geometry']=Point(bus.nodes[n]['x'], bus.nodes[n]['y'])
for e1,e2,k,data in bus.edges(data=True,keys=True):
    data['geometry'] = wkt.loads(data['geometry'])
    data['geometry'] = transform_geometry(data['geometry'])
    data['length_meter'] = data['geometry'].length
    data['time_min'] = data['length_meter'] / 1000
bus.graph['crs'] = local_crs

# convert geometry
for e1,e2,k,data in walk.edges(data=True,keys=True):
    data['geometry'] = wkt.loads(data['geometry'])

for n, d in walk.nodes(data=True):
    point = Point(d['x'], d['y'])
    walk.nodes[n]['geometry']=Point(walk.nodes[n]['x'], walk.nodes[n]['y'])

In [ ]:
for node,data in bus.nodes(data=True):
    data['nodeID'] = node
for node,data in walk.nodes(data=True):
    data['nodeID'] = node

In [ ]:
def nx_to_gdf_custom(net, nodeID='in'):
    nodes = []
    for nid, (n, data) in enumerate(net.nodes(data=True)):
        data[nodeID] = nid
        if 'x' in data and 'y' in data:
            point = Point(data['x'], data['y'])
            nodes.append({**data, 'geometry': point})
        else:
            raise ValueError(f"Node {n} does not have 'x' and 'y' attributes.")
    
    gdf_nodes = gpd.GeoDataFrame(nodes)
    
    edges = []
    for u, v, data in net.edges(data=True):
        if 'geometry' in data:
            geom = data['geometry']
            if not isinstance(geom, (LineString, MultiLineString)):
                raise ValueError(f"Edge ({u}, {v}) has invalid geometry type: {type(geom)}")
        else:
            if 'x' in net.nodes[u] and 'y' in net.nodes[u] and 'x' in net.nodes[v] and 'y' in net.nodes[v]:
                geom = LineString([Point(net.nodes[u]['x'], net.nodes[u]['y']),
                                   Point(net.nodes[v]['x'], net.nodes[v]['y'])])
            else:
                raise ValueError(f"Edge ({u}, {v}) does not have enough information to create geometry.")
        edges.append({**data, 'geometry': geom, 'u': u, 'v': v})
    
    gdf_edges = gpd.GeoDataFrame(edges)
    
    return gdf_nodes, gdf_edges

# Assuming 'bus' is your NetworkX graph
walk_nodes_gdf, walk_edges_gdf = nx_to_gdf_custom(walk)
bus_nodes_gdf, bus_edges_gdf = nx_to_gdf_custom(bus)

# Set the geometry column and CRS
walk_edges_gdf.set_geometry('geometry', inplace=True)
walk_edges_gdf.set_crs(epsg=local_crs, inplace=True)
walk_nodes_gdf.set_crs(epsg=local_crs, inplace=True)

bus_edges_gdf.set_geometry('geometry', inplace=True)
bus_edges_gdf.set_crs(epsg=local_crs, inplace=True)
bus_nodes_gdf.set_crs(epsg=local_crs, inplace=True);

In [ ]:
def transform_multilinestring_to_linestring(geometry):
    if isinstance(geometry, MultiLineString):
        coords = []
        for line in geometry.geoms:  # Use geometry.geoms to iterate over the lines
            coords.extend(line.coords)
        return LineString(coords)
    return geometry

def convert_geometries_to_linestring(graph):
    for u, v, key, data in graph.edges(data=True, keys=True):
        if 'geometry' in data:
            data['geometry'] = transform_multilinestring_to_linestring(data['geometry'])

# Apply the transformation to your graph
convert_geometries_to_linestring(bus)

In [ ]:
trans = gpd.sjoin_nearest(bus_nodes_gdf,walk_nodes_gdf)
node_mapping = dict(zip(trans['nodeID_left'], trans['nodeID_right']))
bus = nx.relabel_nodes(bus, node_mapping)

In [ ]:
inter = walk.copy()
for u, v, key, data in bus.edges(data=True, keys=True):
        inter.add_edge(u, v, key=key, **data)


In [ ]:
for e1,e2,key,data in inter.edges(data=True,keys=True):
    if 'type' not in data:
        data['type'] = 'walk'

In [ ]:
inter_= graphbuilder.convert_list_attr_to_str(inter)
for node,e,data in inter_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
for node,data in inter_.nodes(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
nx.write_graphml(inter_, f"../graphs/{region_name}/{region_name}_inter.graphml")